<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/langgraph_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph

https://qiita.com/isanakamishiro2/items/d14235bf6fe00ae22831

https://github.com/langchain-ai/langgraph/blob/main/examples/agent_executor/force-calling-a-tool-first.ipynb


In [2]:
!pip install --quiet -U langchain langchain_openai tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [14]:
!pip install -U transformers accelerate "exllamav2>=0.0.11" langchain langchainhub duckduckgo-search


In [4]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults, DuckDuckGoSearchRun

search = DuckDuckGoSearchRun(max_results=1)

# Agentに利用するツールのリスト
tools = [search]

In [5]:
from langchain import hub
from langchain.agents import create_json_chat_agent

prompt = hub.pull("hwchase17/react-chat-json")

# Construct the ReAct agent
agent_runnable = create_json_chat_agent(chat_model, tools, prompt)

NameError: name 'chat_model' is not defined

In [8]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

#tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [9]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [11]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00


In [12]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [13]:
tools[0].name

'duckduckgo_search'

In [17]:
from langchain_core.agents import AgentActionMessageLog


def first_agent(inputs):
    action = AgentActionMessageLog(
        # We force call this tool
        tool="duckduckgo_search",
        # We just pass in the `input` key to this tool
        tool_input=inputs["input"],
        log="",
        message_log=[],
    )
    return {"agent_outcome": action}

In [18]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.add_node("first_agent", first_agent)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("first_agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# After the first agent, we want to take an action
workflow.add_edge("first_agent", "action")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [19]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='duckduckgo_search', tool_input='what is the weather in sf', log='', message_log=[])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='duckduckgo_search', tool_input='what is the weather in sf', log='', message_log=[]), "Location: San Francisco, CA Elevation: 39ft Nearby Weather Stations Today Hourly 10-Day Calendar History Wundermap settings Customize Sun 2/18 Mon 2/19 Tue 2/20 Wed 2/21 Thu 2/22 Fri 2/23 Sat... San Francisco Bay Area, CA Heavy Snow in the Central and Southern High Plains; Heavy Rain from the Southern Plains to Tennessee Valley A storm system will produce a few weather hazards as it move across the southern U.S. this weekend. Heavy, accumulating snow is possible in the central and southern High Plains. Feb 19, 2024. The National Weather Service forecast for the greater San Francisco Bay Area for Monday calls for up to 90 percent chance of heavy rain, along with a chance of thunderstorms. Watch ... TUESDAY: Clouds to

In [21]:
result = app.invoke({"input": "what is DataStax", "chat_history": []})

print(result["agent_outcome"].return_values["output"])

DataStax is a company known for its real-time AI solutions and is a leading commercial vendor behind the open-source Apache Cassandra database. It offers AstraDB, a cloud database-as-a-service based on Apache Cassandra. DataStax also works on cutting-edge technologies such as generative AI, vector search, and real-time data innovations.


In [23]:
result = app.invoke({"input": "what is the weather in Tokyo", "chat_history": []})

print(result["agent_outcome"].return_values["output"])

The weather in Tokyo is currently partly cloudy with a temperature of 56°F.


In [31]:
from langchain.globals import set_debug, set_verbose

set_debug(False)
set_verbose(False)

In [33]:
inputs = {"input": "Generate an address book with addresses, names and phone numbers.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='duckduckgo_search', tool_input='Generate an address book with addresses, names and phone numbers.', log='', message_log=[])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='duckduckgo_search', tool_input='Generate an address book with addresses, names and phone numbers.', log='', message_log=[]), "Using Filters to Sort the Google Sheets Contact List Template. At this point, you can choose to add filters to your contact list to sort your list by name. Here are the steps you need to follow: Select all the column headers of your contact list. From the Data menu, select ' Create a Filter '. Alternatively you can click on the 'Create a ... Top 6 FREE Google Sheets Address Book Template [2024] - Ultimate Guide. Google Sheets is a cloud-based spreadsheet program with versatile functions and tools. You can create a robust address book to store contact details such as emails, phone numbers, and physical addresses. Maintaining an address bo

In [34]:
# Define a new graph
workflow2 = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow2.add_node("agent", run_agent)
workflow2.add_node("action", execute_tools)
#workflow.add_node("first_agent", first_agent)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow2.set_entry_point("agent")

# We now add a conditional edge
workflow2.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow2.add_edge("action", "agent")

# After the first agent, we want to take an action
#workflow.add_edge("first_agent", "action")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app2 = workflow2.compile()

In [38]:
inputs = {"input": "Generate an address book having 3 contacts with addresses, names and phone numbers in JSON format. Output shold includes only JSON address list without any response.", "chat_history": []}
for s in app2.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': '{\n  "addressBook": [\n    {\n      "name": "John Doe",\n      "address": "123 Main Street, City, Country",\n      "phone": "123-456-7890"\n    },\n    {\n      "name": "Jane Smith",\n      "address": "456 Oak Avenue, Town, Country",\n      "phone": "987-654-3210"\n    },\n    {\n      "name": "Bob Johnson",\n      "address": "789 Elm Road, Village, Country",\n      "phone": "456-789-0123"\n    }\n  ]\n}\n{\n  "addressBook": [\n    {\n      "name": "John Doe",\n      "address": "123 Main Street, City, Country",\n      "phone": "123-456-7890"\n    },\n    {\n      "name": "Jane Smith",\n      "address": "456 Oak Avenue, Town, Country",\n      "phone": "987-654-3210"\n    },\n    {\n      "name": "Bob Johnson",\n      "address": "789 Elm Road, Village, Country",\n      "phone": "456-789-0123"\n    }\n  ]\n}'}, log='{\n  "addressBook": [\n    {\n      "name": "John Doe",\n      "address": "123 Main Street, City, Country",\n      "pho

In [44]:
result = app.invoke({"input": "Generate a virtual address book having as much unreal contacts with addresses, names and phone numbers as possible in JSON format. Output shold includes only JSON address list without any response.", "chat_history": []})

print(result["agent_outcome"].return_values["output"])

Here is a virtual address book with unreal contacts in JSON format:

```json
{
  "addressBook": [
    {
      "name": "John Doe",
      "address": "123 Main Street, Anytown, USA",
      "phone": "555-1234"
    },
    {
      "name": "Jane Smith",
      "address": "456 Elm Street, Imaginary City, Wonderland",
      "phone": "555-5678"
    },
    {
      "name": "Bob Johnson",
      "address": "789 Oak Avenue, Fantasy Town, Dreamland",
      "phone": "555-9876"
    },
    {
      "name": "Alice Williams",
      "address": "321 Maple Lane, Make-Believe Village, Neverland",
      "phone": "555-4321"
    }
  ]
}
```


https://zenn.dev/tellernovel_inc/articles/7c99563fad1319

# Pydantic + JSON MODE

In [45]:
import json

import tiktoken
from openai import OpenAI
from pydantic import BaseModel, Field
from rich import print

client = OpenAI()

In [55]:
original_prompt = """以下のjson 形式に従って、架空の住所録を作成してください。できるだけ多くの連絡先を出力してください
{
    "addressbook": [
        {
            "name": "名前",
            "location": "住所"
        }
    ]
}
"""

In [53]:
original_prompt = """以下のjson 形式に従って、架空の住所録を作成してください。できるだけ多くの連絡先を出力してください
[
        {
            "name": "名前",
            "location": "住所"
        }
]
"""

In [51]:
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_format={"type": "json_object"},
    temperature=0.0,
    messages=[
        {"role": "system", "content": "あなたは日本の地理を使います"},
        {"role": "user", "content": original_prompt},
    ]
)
print(response.choices[0].message.content)
print(response.usage)

{
    "addressbook": [
        {
            "name": "山田太郎",
            "location": "東京都新宿区西新宿2-8-1"
        },
        {
            "name": "鈴木一郎",
            "location": "大阪府大阪市北区梅田1-1-3"
        },
        {
            "name": "佐藤花子",
            "location": "北海道札幌市中央区北五条西2-5-7"
        },
        {
            "name": "伊藤翔太",
            "location": "愛知県名古屋市中村区名駅3-28-12"
        },
        {
            "name": "高橋美咲",
            "location": "福岡県福岡市博多区博多駅中央街7-10-1"
        },
        {
            "name": "田中健",
            "location": "広島県広島市中区基町12-34"
        },
        {
            "name": "渡辺直美",
            "location": "宮城県仙台市青葉区一番町4-6-1"
        },
        {
            "name": "小林誠",
            "location": "京都府京都市下京区四条通り河原町東入ル真町100"
        },
        {
            "name": "加藤浩二",
            "location": "神奈川県横浜市西区みなとみらい2-3-5"
        },
        {
            "name": "中村悠真",
            "location": "沖縄県那覇市おもろまち4-3-1"
        }
    ]
}

CompletionUsage(completion_tokens=469, prompt_tokens=98, total_tokens=567)

In [56]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    temperature=0.0,
    messages=[
        {"role": "system", "content": "あなたは日本の地理を使います"},
        {"role": "user", "content": original_prompt},
    ]
)
print(response.choices[0].message.content)
print(response.usage)

{
    "addressbook": [
        {
            "name": "田中太郎",
            "location": "東京都渋谷区渋谷1-1-1"
        },
        {
            "name": "山田花子",
            "location": "大阪府大阪市中央区難波5-5-5"
        },
        {
            "name": "鈴木一郎",
            "location": "北海道札幌市中央区北1条西1-1"
        },
        {
            "name": "佐藤美奈子",
            "location": "福岡県福岡市博多区博多1-1-1"
        },
        {
            "name": "伊藤健太",
            "location": "愛知県名古屋市中区栄2-2-2"
        }
    ]
}

CompletionUsage(completion_tokens=228, prompt_tokens=98, total_tokens=326)

In [57]:
response

ChatCompletion(id='chatcmpl-8uw1tkq7fx0XT4gPqg4w0nayy9hKV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "addressbook": [\n        {\n            "name": "田中太郎",\n            "location": "東京都渋谷区渋谷1-1-1"\n        },\n        {\n            "name": "山田花子",\n            "location": "大阪府大阪市中央区難波5-5-5"\n        },\n        {\n            "name": "鈴木一郎",\n            "location": "北海道札幌市中央区北1条西1-1"\n        },\n        {\n            "name": "佐藤美奈子",\n            "location": "福岡県福岡市博多区博多1-1-1"\n        },\n        {\n            "name": "伊藤健太",\n            "location": "愛知県名古屋市中区栄2-2-2"\n        }\n    ]\n}', role='assistant', function_call=None, tool_calls=None))], created=1708580125, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_56b2abeda9', usage=CompletionUsage(completion_tokens=228, prompt_tokens=98, total_tokens=326))

In [62]:
import json

jobj = json.loads(response.choices[0].message.content)

In [63]:
jobj

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '佐藤美奈子', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'}]}

In [65]:
!pip install dictknife

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 2.3 MB/s eta 0:00:00


In [67]:
from dictknife import deepmerge

j3 = deepmerge(jobj, jobj)

In [68]:
j3

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '佐藤美奈子', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'}]}

In [69]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    temperature=0.0,
    messages=[
        {"role": "system", "content": "あなたは日本の地理を使います"},
        {"role": "user", "content": original_prompt},
    ]
)
print(response2.choices[0].message.content)
print(response2.usage)

{
    "addressbook": [
        {
            "name": "田中太郎",
            "location": "東京都渋谷区神南1-2-3"
        },
        {
            "name": "山田花子",
            "location": "大阪府大阪市中央区難波5-6-7"
        },
        {
            "name": "鈴木一郎",
            "location": "北海道札幌市中央区北1-2-3"
        },
        {
            "name": "佐藤明美",
            "location": "福岡県福岡市博多区博多1-2-3"
        },
        {
            "name": "伊藤健太",
            "location": "愛知県名古屋市中区栄2-3-4"
        }
    ]
}

CompletionUsage(completion_tokens=224, prompt_tokens=98, total_tokens=322)

In [70]:
j3 = deepmerge(jobj, json.loads(response2.choices[0].message.content))

In [71]:
j3

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '佐藤美奈子', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
  {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'}]}

In [80]:
def submit(jobj, original_prompt):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    temperature=0.0,
    messages=[
        {"role": "system", "content": "あなたは日本の地理を使います"},
        {"role": "user", "content": original_prompt},
    ]
  )
  retobj = deepmerge(jobj, json.loads(response.choices[0].message.content))
  return retobj

In [85]:
def start(num=15):
  original_prompt = """以下のjson 形式に従って、架空の住所録を作成してください。できるだけ多くの連絡先を出力してください
{
    "addressbook": [
        {
            "name": "名前",
            "location": "住所"
        }
    ]
}
"""
  obj = {}
  while True:
    obj = submit(obj, original_prompt)
    print(len(obj['addressbook']) )
    if len(obj['addressbook']) > num:
      break
  return obj


In [96]:
def generate(element_name, json_sample, num=15):
  original_prompt = """以下のjson 形式に従って、架空のデータを作成してください。できるだけ多くのデータを出力してください
{{
    "{0}": [
        {1}
    ]
}}
""".format(element_name, json_sample)

  obj = {}
  while True:
    obj = submit(obj, original_prompt)
    print(len(obj[element_name]) )
    if len(obj[element_name]) > num:
      break
  return obj

In [97]:
json_sample = """
 {
   "name": "名前",
   "location": "住所"
}
"""
element_name = "contacts"
obj = generate(element_name, json_sample, 3)
obj

9

{'contacts': [{'name': '田中太郎', 'location': '東京都渋谷区'},
  {'name': '山田花子', 'location': '大阪府大阪市'},
  {'name': '鈴木一郎', 'location': '北海道札幌市'},
  {'name': '佐藤明美', 'location': '福岡県福岡市'},
  {'name': '高橋健太', 'location': '愛知県名古屋市'},
  {'name': '伊藤さちこ', 'location': '神奈川県横浜市'},
  {'name': '中村隆', 'location': '京都府京都市'},
  {'name': '小林美和', 'location': '広島県広島市'},
  {'name': '加藤健太郎', 'location': '沖縄県那覇市'}]}

In [99]:
json_sample = """
 {
   "name": "名前",
   "location": "住所"
}
"""
element_name = "contacts"
obj = generate(element_name, json_sample, 3)
obj

10

{'contacts': [{'name': '田中太郎', 'location': '東京都渋谷区'},
  {'name': '山田花子', 'location': '大阪府大阪市'},
  {'name': '鈴木一郎', 'location': '北海道札幌市'},
  {'name': '佐藤次郎', 'location': '福岡県福岡市'},
  {'name': '伊藤三郎', 'location': '愛知県名古屋市'},
  {'name': '中村四郎', 'location': '神奈川県横浜市'},
  {'name': '加藤五郎', 'location': '京都府京都市'},
  {'name': '山口六郎', 'location': '広島県広島市'},
  {'name': '小林七郎', 'location': '岡山県岡山市'},
  {'name': '斎藤八郎', 'location': '沖縄県那覇市'}]}

In [100]:
element_name = "books"
json_sample = """
 {
   "name": "書名",
   "author": "著者名"
}
"""
obj = generate(element_name, json_sample, 3)
obj

10

{'books': [{'name': '火山の謎', 'author': '山田太郎'},
  {'name': '東京タワーの秘密', 'author': '鈴木花子'},
  {'name': '北海道の自然', 'author': '田中健太'},
  {'name': '沖縄の歴史', 'author': '佐藤美智子'},
  {'name': '富士山の神話', 'author': '中村一郎'},
  {'name': '京都の文化', 'author': '鈴木雅美'},
  {'name': '大阪の食文化', 'author': '田中太郎'},
  {'name': '広島の平和', 'author': '山田花子'},
  {'name': '名古屋の町並み', 'author': '佐藤健太'},
  {'name': '神戸の港町', 'author': '中村美智子'}]}

In [101]:
element_name = "novels"
json_sample = """
 {
   "name": "題名",
   "author": "著者名",
   "genre": "ジャンル",
   "opening": "冒頭抜粋",
   "summary": "概要"
}
"""
obj = generate(element_name, json_sample, 3)
obj

5

{'novels': [{'name': '夜の果ての街',
   'author': '田中太郎',
   'genre': 'ミステリー',
   'opening': '深夜の街は静かで、ただひときわ輝く明かりがあった。',
   'summary': '主人公が夜の街で起こる連続殺人事件の謎を解き明かすミステリー小説。'},
  {'name': '遥かなる冒険',
   'author': '山田花子',
   'genre': '冒険',
   'opening': '広大な海原に漂う船の上で、主人公は新たな大陸を目指していた。',
   'summary': '主人公が仲間たちとともに未知の大陸を目指し、様々な困難に立ち向かう冒険小説。'},
  {'name': '愛と友情の絆',
   'author': '鈴木花子',
   'genre': '恋愛',
   'opening': '幼なじみの二人は、ずっと一緒に過ごしてきた。',
   'summary': '幼なじみの二人が愛と友情の絆を深めながら、さまざまな試練に立ち向かう恋愛小説。'},
  {'name': '未来都市の謎',
   'author': '佐藤太郎',
   'genre': 'SF',
   'opening': '未来の都市は高層ビルが立ち並び、空を飛ぶ車が行き交っていた。',
   'summary': '未来の都市で起こる謎の事件を追う主人公の活躍を描いたSF小説。'},
  {'name': '時空を超える旅',
   'author': '田中花子',
   'genre': 'ファンタジー',
   'opening': '古びた時計台の中で、主人公は時空を超える旅に出ることを決意した。',
   'summary': '主人公が時空を超え、異世界での冒険に挑むファンタジー小説。'}]}

In [93]:
original_prompt = """以下のjson 形式に従って、架空のデータを作成してください。できるだけ多くのデータを出力してください
{{
    {0}: [
        {1}
    ]
}}
""".format("element_name", "json_sample")
original_prompt

'以下のjson 形式に従って、架空のデータを作成してください。できるだけ多くのデータを出力してください\n{\n    element_name: [\n        json_sample\n    ]\n}\n'

In [86]:
obj = start(30)
obj

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'}
    ]
}

...

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
        {'name': '佐藤美智子', 'location': '福岡県福岡市博多区博多駅前1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '高橋健太', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '佐藤美代', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-2-3'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
        {'name': '佐藤美智子', 'location': '福岡県福岡市博多区博多駅前1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '高橋健太', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '佐藤美代', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-2-3'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
        {'name': '佐藤美智子', 'location': '福岡県福岡市博多区博多駅前1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '高橋健太', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '佐藤美代', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-2-3'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
        {'name': '佐藤美智子', 'location': '福岡県福岡市博多区博多駅前1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-2-2'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '高橋健太', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '佐藤美代', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区東1-2-3'}
    ]
}

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
  {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-2-3'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-4-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-2'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区中洲5-6-7'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-2-1'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
  {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '佐藤美代', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '佐藤美智子', 'location': '福岡県福岡市博多区博多駅前1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'},
  {'nam

In [73]:
newobj = submit(j3)

In [74]:
newobj

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '佐藤美奈子', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-2-2'},
  {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
  {'name': '佐藤明美', 'location': '愛知県名古屋市中区栄2-3-4'},
  {'name': '伊藤健太', 'location': '福岡県福岡市博多区博多1-2-3'}]}

In [76]:
len(newobj['addressbook'])

12

In [78]:
obj = {}
while True:
  obj = submit(obj)
  print(obj)
  if len(obj['addressbook']) > 15:
    break

obj

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-4-5'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-4-5'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-4-5'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区東1-2-3'}
    ]
}

{
    'addressbook': [
        {'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-4-5'},
        {'name': '佐藤明美', 'location': '福岡県福岡市博多区東1-2-3'},
        {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
        {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
        {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
        {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'}
    ]
}

{'addressbook': [{'name': '田中太郎', 'location': '東京都渋谷区神南1-2-3'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-6-7'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-2-3'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-2-3'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄2-3-4'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波1-1-1'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西2丁目'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多駅前1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-6-1'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1-1-1'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区博多1-1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-4-5'},
  {'name': '佐藤明美', 'location': '福岡県福岡市博多区東1-2-3'},
  {'name': '田中太郎', 'location': '東京都渋谷区渋谷1-1-1'},
  {'name': '山田花子', 'location': '大阪府大阪市中央区難波5-5-5'},
  {'name': '鈴木一郎', 'location': '北海道札幌市中央区北1条西1-1'},
  {'name': '伊藤健太', 'location': '愛知県名古屋市中区栄3-3-3'}]}

In [9]:
!mkdir openchat-3.5-1210-GPTQ
!huggingface-cli download TheBloke/openchat-3.5-1210-GPTQ --local-dir openchat-3.5-1210-GPTQ --local-dir-use-symlinks False

mkdir: cannot create directory ‘openchat-3.5-1210-GPTQ’: File exists
Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 11 files:   0% 0/11 [00:00<?, ?it/s]downloading https://huggingface.co/TheBloke/openchat-3.5-1210-GPTQ/resolve/b3ea362e1007bc06211adcbbb4cfab0a132dcd6d/quantize_config.json to /root/.cache/huggingface/hub/tmp5bymszwc
downloading https://huggingface.co/TheBloke/openchat-3.5-1210-GPTQ/resolve/b3ea362e1007bc06211adcbbb4cfab0a132dcd6d/config.json to /root/.cache/huggingface/hub/tmptblqyqaf
downloading https://huggingface.co/TheBloke/openchat-3.5-1210-GPTQ/resolve/b3ea362e1007bc06211adcbbb4cfab0a132dcd6d/openchat.json to /root/.cache/huggingface/hub/tmp47_r19zq
downloading https://huggingface.co/TheBloke/openchat-3.5-1210-GPTQ/resolve/b3ea362e1007bc06211adcbbb4cfab0a132dcd6d/added_tokens.json to /root/.cache/huggingface/hub/tmp_rfs7g7e
downloadi